In [ ]:
## Correct Code
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib

# Load your data
data = pd.read_excel("Student_Employability_dataset_2025.xlsx")

# Drop 'Name_of_Student' column if exists
data = data.drop(['Name_of_Student'], axis=1, errors='ignore')

# Encode target variable 'CLASS'
le = LabelEncoder()
data['CLASS'] = le.fit_transform(data['CLASS'])
print(f"Encoded classes: {dict(zip(le.classes_, le.transform(le.classes_)))}")

# Feature selection
X = data.drop(['CLASS'], axis=1)
y = data['CLASS']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)

# Define SVM model and parameters
svm_model = SVC(probability=True, random_state=42)
svm_params = {
    'C': [0.1, 1, 10],
    'gamma': [0.1, 1, 10],
    'kernel': ['rbf']
}

# GridSearchCV for hyperparameter tuning
grid = GridSearchCV(svm_model, svm_params, cv=5, scoring='accuracy')
grid.fit(X_train_scaled, y_train_smote)

# Best model
best_model = grid.best_estimator_
print("Best parameters:", grid.best_params_)

# Evaluate on test set
y_pred = best_model.predict(X_test_scaled)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Save model and scaler
joblib.dump(best_model, 'employability_predictor.pkl')
joblib.dump(scaler, 'scaler.pkl')


In [ ]:
data.sample(10)